In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import inception_v3
from torch.utils.data.dataset import random_split
import torchvision
from going_modular.engine import train
from going_modular.engine import train


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# Definir rutas de los directorios
data_dir = "./dataset/chest_xray/"
train_dir = data_dir + 'train/'
test_dir = data_dir + 'test/'

transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                          [0.229, 0.224, 0.225]) ])

# Dataset y DataLoader
train_dataset = ImageFolder(root=train_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)
train_size = int(0.85 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [9]:
# Instanciar el modelo base (InceptionV3 pre-entrenado)
base_model = inception_v3(weights="DEFAULT").to(device)
base_model.aux_logits = False

for parameter in base_model.parameters():
    parameter.requires_grad = False

base_model.fc = nn.Sequential(
    nn.Linear(2048, 2).to(device)
)

# # Definir optimizador y función de pérdida
optimizer = optim.Adamax(base_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [11]:

train(model=base_model, train_dataloader=train_loader, test_dataloader=test_loader,
      optimizer=optimizer, loss_fn=criterion, device=device, epochs=10)


 10%|█         | 1/10 [01:05<09:45, 65.02s/it]

Epoch: 1 | train_loss: 0.2684 | train_acc: 0.8955 | test_loss: 0.5322 | test_acc: 0.7422


 10%|█         | 1/10 [01:47<16:10, 107.88s/it]


KeyboardInterrupt: 

################### aui

In [16]:
# Define tu modelo en PyTorch
class CustomModel(nn.Module):
    def __init__(self, base_model, num_classes):
        super(CustomModel, self).__init__()
        self.base_model = base_model
        self.batch_norm = nn.BatchNorm1d(num_features=2, )
        self.dropout1 = nn.Dropout(p=0.45)
        self.fc1 = nn.Linear(2, 220)
        self.dropout2 = nn.Dropout(p=0.25)
        self.fc2 = nn.Linear(220, 60)
        self.fc3 = nn.Linear(60, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.base_model(x)
        x = self.batch_norm(x)
        x = self.dropout1(x)
        x = self.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x

model_2 = CustomModel(base_model, 2)

In [17]:

train(model=base_model, train_dataloader=train_loader, test_dataloader=test_loader,
      optimizer=optimizer, loss_fn=criterion, device=device, epochs=10)

 10%|█         | 1/10 [01:06<10:01, 66.87s/it]

Epoch: 1 | train_loss: 0.2251 | train_acc: 0.9137 | test_loss: 0.5915 | test_acc: 0.7281


 20%|██        | 2/10 [02:15<09:05, 68.17s/it]

Epoch: 2 | train_loss: 0.2133 | train_acc: 0.9185 | test_loss: 0.5861 | test_acc: 0.7391


 30%|███       | 3/10 [03:24<07:59, 68.54s/it]

...

 90%|█████████ | 9/10 [10:10<01:06, 66.60s/it]

Epoch: 9 | train_loss: 0.1834 | train_acc: 0.9283 | test_loss: 0.6002 | test_acc: 0.7578


100%|██████████| 10/10 [11:13<00:00, 67.35s/it]

Epoch: 10 | train_loss: 0.1781 | train_acc: 0.9263 | test_loss: 0.5426 | test_acc: 0.7766


{'train_loss': [0.22514717157581726,
  0.21325294925368948,
  0.21059344687479006,
  0.20054089830076094,
  0.19347374746910959,
  0.19158323508479613,
  0.1918591723489247,
  0.18911707476317455,
  0.1833565483847968,
  0.17807368746549962],
 'train_acc': [0.9137219636055861,
  0.9184696360558612,
  0.9157188954718578,
  0.9237859712230215,
  0.9267350825222175,
  0.9251877909437156,
  0.9258358019466779,
  0.9240372407955988,
  0.928335272958104,
  0.9262854422344478],
 'test_loss': [0.5915165395475924,
  0.5861499624326825,
  0.48836138248443606,
  0.5736527337692678,
  0.52594073433429,
  0.5293496836908161,
  0.5393739167600871,
  0.5361750861629844,
  0.60018116873689,
  0.5425746716558933],
 'test_acc': [0.728125,
  0.7390625,
  0.7875,
  0.7609375,
  0.775,
  0.7765625,
  0.771875,
  0.7796875,
  0.7578125,
  0.7765625]}